vou montar meu roadmap primeiramente e depois eu organizo melhor

Como o luqueta pediu para apresentar a estratégia de "Vector Quantization" utilizando o algoritmo do kmeans, vou primeiro explicar o que o é vector quantization; 
<br>depois explico o que é kmeans; 
<br>depois explico o porque usar kmeans na técnica de vector quantization; 
<br>o que nós ganhamos com isso x o que perdemos; 
<br>o que é a taxa de compressão (benefícios x malefícios); 
<br>exemplo rodando; 

comparar os gráficos de loss (não fiz ainda) 


---

Vector quantization é uma estratégia para quantizar vetores (rsrsr a sério? kkk), mas sim o intuito do algoritmo é achar uma forma de "comprir o vetor", mas mantendo claro a sua informação, intuivamente, é como pensarmos em achar uma forma de representar o mesmo vetor e suas informações só que em um espaço de memória menor. 

E por que isso é importante aqui? Pois quando estamos falando principalmente de imagens que carregam por si só muita informação em um único píxel, partimos do pressuposto de que é muito importante para nós usarmos o máximo de informações possíveis para qualquer processamento/utilização dessa imagem em si. Porém, infelizmente, o mundo não é um morango e precisamos achar uma forma de diminuir a quantidade de memória que usamos, porque caso contrário, não teríamos espaços suficientes e eu digo isso não só com memória para armazenar a imagem em si, mas em memória latente mesmo... digo, de processamento. 

Sobre o algoritmo kmeans, já estamos cansados de saber que é um algoritmo, em grosso modo, focado em clusterizar informações, então pensem comigo... eu tenho vários vetores em uma imagem, certo? E se eu achar uma forma de pelo menos minimizar o espaço vetorial que eles estão inseridos, ou seja, e se eu achar uma forma de representar a minha informação em um espaço reduzido? Isso parece interessante, ne? Só que temos que lembrar que cada píxel da imagem pode ser muito importante, então a intuição por trás aqui, é achar formas de representar os meus píxeis importantes em outros píxeis, ou no nosso caso, em aglomerados de píxeis, isso é, nossos centróides do Kmeans. 

Para isso o Kmeans vai calcular para cada ponto a menor distância euclidiana entre pontos, ou seja, imagine que nós falamos para o algoritmo que eu quero que no meu espaço vetorial exista 10 centróides, nós vamos inicializar esses 10 pontos de forma aleatória, calcular a distância de cada ponto a esses centróides e ir encontrando qual é a menor distrância euclidiana desses pontos aos centróides, depois ir recalculando cada centróide como a média dos vetores atribuídos a esse cluster, pegando o novo centróide desses vetores nesse cluster. E vamos ponderar isso com o Erro Médio Quadrático, intuitivamente é como se eu falasse:

**"Olhe para esse vetor, agora olhe para o centróide que ele esta inserido, calcule o Erro Médio Quadrático dele, com isso, o que eu quero é que você miniminize esse erro, isso é, buscar sempre a menor diferença entre o vetor original e onde ele está inserido agora, só que sem aumentar o K, ou seja, sem criar novos clusters só para encontrar um erro baixo rsrs"**

Então com isso, eu acho uma forma mais eficiente em conseguir processar grandes volumes de imagem, pois estaria diminuindo a quantidade de bits necessários para cada uma. 

Exemplo:
<br>Tamanho orignal (bits): N * d * b 
<br>Sendo N a quantidade de vetores, d a dimensionalidade de cada vetor, b o numéro de bits para armazenar cada componente, isso é, por exemplo eu falar que eu vou deixar ter 16bf de armazenamento para esse vetor.

<br>E o tamanho comprimido (bits): K * d * b + N * log2 K
<br>Sendo esse K o número de codewords no codebook, o que é como se fosse a quantidade de "informações codadas", ou seja, a quantidade de clusters que queremos kkkk

E a taxa de compressão, que é a escada que vamos usar para limitar o quanto a minha imagem pode perder de informação é dada pela formula:

TamanhoOriginal / TamanhoComprimido

Ou seja, temos que quanto maior o meu K, maior qualidade, pois eu vou ter menos informações quantizadas, mas o codebook vai crescer muito e eu estaria reduzindo a compressão que estamos fazendo, se associoarmos ao Kmeans, é como se eu tivesse falando que tehno mais clusters, logo eu tenho mais informações, porém o custo é menor.
O mesmo vale para o contrário. 

**E aqui entra um conceito importante: o K_max!**

O K_max é basicamente o valor máximo de K (número de clusters) que eu posso usar mantendo ainda uma taxa de compressão aceitável. No nosso caso, queremos garantir pelo menos 50% de compressão, então o K_max é o maior K onde:

TamanhoComprimido ≤ 0.5 × TamanhoOriginal

Ou seja: K * d * b + N * log2(K) ≤ 0.5 × (N * d * b)

Isso é importante porque não adianta eu ter K=10000 clusters se isso vai fazer minha "compressão" ficar maior que a imagem original rsrs. O K_max me dá o limite teórico onde ainda vale a pena usar a vector quantization. Na prática, a gente geralmente usa valores bem menores que K_max porque o algoritmo fica muito lento e não compensa o ganho de qualidade.

Por exemplo, para uma imagem 256×256, o K_max teórico pode ser 2000+, mas na prática usamos K entre 10-100 para ter um bom trade-off entre qualidade e eficiência computacional.

## Pixel-VQ vs VQ 2×2: Qual estratégia é melhor?

Agora que entendemos o conceito, vamos comparar as duas estratégias principais:

**Pixel-VQ (Quantização por Pixel):**
- Cada pixel RGB vira um vetor 3D → [R, G, B]
- Clusteriza no espaço 3D (mais simples)
- Cada pixel é substituído pelo centróide mais próximo
- Flexibilidade: cada pixel é tratado independentemente (o que não podemos falar que é bom, pois perdemos a capacidade de analisar os vizinhos)

**VQ 2×2 (Quantização por Janelas 2×2):**
- Cada bloco 2×2 vira um vetor 12D → [R₁,G₁,B₁, R₂,G₂,B₂, R₃,G₃,B₃, R₄,G₄,B₄]
- Clusteriza no espaço 12D (mais complexo)
- Cada bloco inteiro é substituído pelo centróide mais próximo
- Preserva relações espaciais: considera pixels vizinhos juntos

## Analisando os resultados:

Olhando pro gráfico da loss de reconstrução, conseguimos ver algumas coisas bem interessantes:

1. **Convergência mais rápida:** A estratégia **Pixel-VQ (azul)** converge muito mais rápido para erro zero. Com poucos clusters (K ≈ 200), ela já atinge MSE praticamente zero.

2. **Comportamento da VQ 2×2:** A estratégia **VQ 2×2 (laranja)** precisa de mais clusters para atingir a mesma qualidade, convergindo mais lentamente.

3. **Eficiência computacional:** Para o mesmo nível de qualidade, Pixel-VQ precisa de menos clusters, sendo mais eficiente.

## Conclusão: Qual é melhor?

**Para esta imagem específica, a estratégia Pixel-VQ se mostrou superior**, pois:

✅ **Converge mais rápido:** Atinge erro zero com menos clusters
✅ **Mais eficiente:** Menos clusters = menos processamento
✅ **Melhor compressão:** Mesmo resultado com menor K

**Mas por que isso acontece?**

A explicação está na natureza dos dados:
- **Pixel-VQ** é mais direta: agrupa cores similares independente da posição
- **VQ 2×2** é mais restritiva: precisa agrupar blocos inteiros que sejam similares
- Para imagens com variações suaves de cor, agrupar pixels individuais é mais eficiente
- VQ 2×2 seria melhor para imagens com texturas repetitivas ou padrões estruturados

**Trade-off importante:** Embora Pixel-VQ tenha sido melhor aqui, VQ 2×2 preserva melhor as estruturas espaciais da imagem, sendo preferível quando queremos manter texturas e bordas mais definidas, mesmo que precise de mais clusters.

Na prática, a escolha depende do tipo de imagem e do que você quer preservar: detalhes de cor (Pixel-VQ) ou estruturas espaciais (VQ 2×2).
